# Snowpark ML Ops: Model Registry
* Models are stored as first-class schema level objects
* Models can be found and used by others

## Main objects
Snowpark model registry API has three main objects:
* `snowflake.ml.registry.Registry` - To manage models within a schema
* `snowflake.ml.model.Model` - To represent models
* `snowflake.ml.model.ModelVersion` - To represent a specific version of a model

## Supported Model Types
Currently supported types of models:
* Snowpark ML Modeling
* scikit-learn
* XGBoost
* PyTorch
* TensorFlow
* MLFlow PyFunc
* HuggingFace pipeline

## Python Environment

### Conda environment.yml file content
```yaml
name: snowpark
channels:
  - snowflake
dependencies:
  - python=3.9
  - openssl=3.0.9
  - conda-forge::charset-normalizer
  - snowflake-ml-python
  - pip
  - pip:
      - "-r requirements.txt"
```

### requirements.txt file content
```
snowflake-snowpark-python[pandas]
tomli
toml
pytest
snowflake-vcrpy @ git+https://github.com/Snowflake-Labs/snowflake-vcrpy.git@v0.1.1
packaging
```

## Creating Snowpark Session

In [34]:
from os import environ
import logging
import warnings

warnings.filterwarnings("ignore")

log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
logging.getLogger('snowflake.connector').setLevel(logging.ERROR)

In [35]:
def get_env_var_config() -> dict:
    """
    Returns a dictionary of the connection parameters using the CLI
    environment variables.
    """
    try:
        return {
            "user": environ["SNOWSQL_USER"],
            "password": environ["SNOWSQL_PWD"],
            "account": environ["SNOWSQL_ACCOUNT"],
            "role": environ["SNOWSQL_ROLE"],
            "warehouse": "snowpark_medium_wh",
            "database": environ["SNOWSQL_DATABASE"],
            "schema": environ["SNOWSQL_SCHEMA"],
        }
    except KeyError as exc:
        raise KeyError(
            "ERROR: Environment variable for Snowflake Connection not found. "
            + "Please set the SNOWSQL_* environment variables"
        ) from exc

In [36]:
from snowflake.snowpark import Session

session = Session.builder.configs(get_env_var_config()).create()

INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 1.11.1,
"python.version" : 3.9.17,
"python.connector.version" : 3.5.0,
"python.connector.session.id" : 2122392449232918,
"os.name" : Darwin



In [37]:
# Make sure these two are present otherwise
# you will get an 'object not found or cannot be operated on' error
DATABASE_NAME = "MACHINE_LEARNING"
SCHEMA_NAME = "MODEL_REGISTRY"

In [38]:
# create a clean schema for model registry
session.sql(f"CREATE OR REPLACE SCHEMA {DATABASE_NAME}.{SCHEMA_NAME}").collect()

# But that's only for model registry, data is present in environ['SNOWSQL_SCHEMA']
session.sql(f"USE SCHEMA {environ['SNOWSQL_DATABASE']}.{environ['SNOWSQL_SCHEMA']}").collect()

[Row(status='Statement executed successfully.')]

## Opening A Registry

In [39]:
from snowflake.ml.registry import Registry

reg = Registry(session=session, database_name=DATABASE_NAME, schema_name=SCHEMA_NAME)

## Training A Model: RandomForestClassifier V1

### Prepare Training Data

In [40]:
log.debug("Reading input table with training data")
training_input_df = session.table('titanic_training')

log.info("Dropping non impact columns")
v1_drop_cols = ['PassengerId', 'Name', 'Ticket', 'Embarked', 'Sex', 'Age', 'Cabin']
v1_training_df = training_input_df.drop(v1_drop_cols)
v1_training_df.show()

INFO:__main__:Dropping non impact columns


-------------------------------------------------------
|"SURVIVED"  |"PCLASS"  |"SIBSP"  |"PARCH"  |"FARE"   |
-------------------------------------------------------
|0           |3         |1        |0        |7.25     |
|1           |1         |1        |0        |71.2833  |
|1           |3         |0        |0        |7.925    |
|1           |1         |1        |0        |53.1     |
|0           |3         |0        |0        |8.05     |
|0           |3         |0        |0        |8.4583   |
|0           |1         |0        |0        |51.8625  |
|0           |3         |3        |1        |21.075   |
|1           |3         |0        |2        |11.1333  |
|1           |2         |1        |0        |30.0708  |
-------------------------------------------------------



### Prepare Test Data

In [41]:
log.debug("Reading input table with training data")
test_input_df = session.table('titanic_test')

log.info("Dropping non impact columns")
v1_test_df = test_input_df.drop(v1_drop_cols)
v1_test_df.show()

INFO:__main__:Dropping non impact columns


-------------------------------------------------------
|"SURVIVED"  |"PCLASS"  |"SIBSP"  |"PARCH"  |"FARE"   |
-------------------------------------------------------
|0           |3         |1        |0        |7.25     |
|1           |1         |1        |0        |71.2833  |
|1           |3         |0        |0        |7.925    |
|1           |1         |1        |0        |53.1     |
|0           |3         |0        |0        |8.05     |
|0           |3         |0        |0        |8.4583   |
|0           |1         |0        |0        |51.8625  |
|0           |3         |3        |1        |21.075   |
|1           |3         |0        |2        |11.1333  |
|1           |2         |1        |0        |30.0708  |
-------------------------------------------------------



### Train the Model

In [42]:
from snowflake.ml.modeling.ensemble import RandomForestClassifier

FEATURE_COLS = ["PCLASS", "SIBSP", "PARCH", "FARE"]
LABEL_COLS = ["SURVIVED"]
OUTPUT_COLS = ["PREDICTIONS"]

v1_rf = RandomForestClassifier(
    input_cols=FEATURE_COLS,
    label_cols=LABEL_COLS,
    output_cols=OUTPUT_COLS
)

v1_rf.fit(v1_training_df)

In [43]:
# Use the model to make predictions.
v1_rf_training_prediction_df = v1_rf.predict(v1_training_df)
v1_rf_training_prediction_df[LABEL_COLS[0], OUTPUT_COLS[0]].show()

------------------------------
|"SURVIVED"  |"PREDICTIONS"  |
------------------------------
|0           |0              |
|1           |1              |
|1           |0              |
|1           |1              |
|0           |0              |
|0           |0              |
|0           |0              |
|0           |0              |
|1           |1              |
|1           |0              |
------------------------------



### Calculate Prediction Accuracy on Training Data

In [44]:
from snowflake.ml.modeling.metrics import accuracy_score, confusion_matrix

v1_rf_training_accuracy_score = accuracy_score(
    df=v1_rf_training_prediction_df,
    y_true_col_names=LABEL_COLS,
    y_pred_col_names=OUTPUT_COLS
)
log.info(f"RandomForestClassifier v1 accuracy_score on training data: {v1_rf_training_accuracy_score}")

v1_rf_training_confusion_matrix = confusion_matrix(
    df=v1_rf_training_prediction_df,
    y_true_col_name=LABEL_COLS[0],
    y_pred_col_name=OUTPUT_COLS[0]
)
log.info(f"RandomForestClassifier v1 confusion_matrix on training data: {v1_rf_training_confusion_matrix}")

INFO:__main__:RandomForestClassifier v1 accuracy_score on training data: 0.836139
INFO:__main__:RandomForestClassifier v1 confusion_matrix on training data: [[496.  53.]
 [ 93. 249.]]


## Logging RandomForestClassifier V1 to Model Registry
Mandatory arguments:
* `model`: It's the serializable/pickleable Python model object
* `model_name`: The name of the model (can't be changed)
* `version_name`: A string with the version of the model. Quick note:- `version_name="1"` is not supported but `version_name="v1"` is.


Other important info:
* The combination of `model_name` and `version_name` must be unique in the schema.
* The `model_name` cannot have dash (-) in it.
* Sometimes it's necessary to mention Python dependenies through the `conda_dependencies` or `pip_requirements` or both arguments.

In [45]:
mv = reg.log_model(
    v1_rf,
    model_name="Titanic_RF",
    version_name="v1",
    pip_requirements=["packaging"],
    comment="Very simple RF model to setup baseline",
    metrics={
        "training_accuracy_score": v1_rf_training_accuracy_score,
        "training_confusion_matrix": v1_rf_training_confusion_matrix.tolist()
    },
    sample_input_data=v1_training_df
)

INFO:snowflake.ml.registry._manager.model_manager:Start packaging and uploading your model. It might take some time based on the size of the model.
INFO:absl:Model signatures are auto inferred as:

{'predict': ModelSignature(
                    inputs=[
                        FeatureSpec(dtype=DataType.INT8, name='PCLASS'),
		FeatureSpec(dtype=DataType.INT8, name='SIBSP'),
		FeatureSpec(dtype=DataType.INT8, name='PARCH'),
		FeatureSpec(dtype=DataType.DOUBLE, name='FARE')
                    ],
                    outputs=[
                        FeatureSpec(dtype=DataType.INT8, name='PCLASS'),
		FeatureSpec(dtype=DataType.INT8, name='SIBSP'),
		FeatureSpec(dtype=DataType.INT8, name='PARCH'),
		FeatureSpec(dtype=DataType.DOUBLE, name='FARE'),
		FeatureSpec(dtype=DataType.INT8, name='PREDICTIONS')
                    ]
                ), 'predict_log_proba': ModelSignature(
                    inputs=[
                        FeatureSpec(dtype=DataType.INT8, name='PCLASS'),
		FeatureS

## Training Another Model: LinearSVC

In [46]:
from snowflake.ml.modeling.svm import LinearSVC

FEATURE_COLS = ["PCLASS", "SIBSP", "PARCH", "FARE"]
LABEL_COLS = ["SURVIVED"]
OUTPUT_COLS = ["PREDICTIONS"]

v1_lsvc = LinearSVC(
    input_cols=FEATURE_COLS,
    label_cols=LABEL_COLS,
    output_cols=OUTPUT_COLS,
    dual=False
)

v1_lsvc.fit(v1_training_df)

In [47]:
# Use the model to make predictions.
v1_lsvc_training_prediction_df = v1_lsvc.predict(v1_training_df)
v1_lsvc_training_prediction_df[LABEL_COLS[0], OUTPUT_COLS[0]].show()

------------------------------
|"SURVIVED"  |"PREDICTIONS"  |
------------------------------
|0           |0              |
|1           |1              |
|1           |0              |
|1           |1              |
|0           |0              |
|0           |0              |
|0           |1              |
|0           |0              |
|1           |0              |
|1           |0              |
------------------------------



### Calculate Prediction Accuracy on Training Data

In [48]:
from snowflake.ml.modeling.metrics import accuracy_score, confusion_matrix
v1_lsvc_training_accuracy_score = accuracy_score(
    df=v1_lsvc_training_prediction_df,
    y_true_col_names=LABEL_COLS,
    y_pred_col_names=OUTPUT_COLS
)
log.info(f"LinearSVC v1 accuracy_score on training data: {v1_lsvc_training_accuracy_score}")

v1_lsvc_training_confusion_matrix = confusion_matrix(
    df=v1_lsvc_training_prediction_df,
    y_true_col_name=LABEL_COLS[0],
    y_pred_col_name=OUTPUT_COLS[0]
)
log.info(f"LinearSVC v1 confusion_matrix on training data: {v1_lsvc_training_confusion_matrix}")

INFO:__main__:LinearSVC v1 accuracy_score on training data: 0.687991
INFO:__main__:LinearSVC v1 confusion_matrix on training data: [[461.  88.]
 [190. 152.]]


## Logging LinearSVC V1 to Model Registry

In [49]:
mv = reg.log_model(
    v1_lsvc,
    model_name="Titanic_LinearSVC",
    version_name="v1",
    pip_requirements=["packaging"],
    comment="Very simple LinearSVC model to setup baseline",
    metrics={
        "training_accuracy_score": v1_lsvc_training_accuracy_score,
        "training_confusion_matrix": v1_lsvc_training_confusion_matrix.tolist()
    },
    sample_input_data=v1_training_df
)

INFO:snowflake.ml.registry._manager.model_manager:Start packaging and uploading your model. It might take some time based on the size of the model.
INFO:absl:Model signatures are auto inferred as:

{'predict': ModelSignature(
                    inputs=[
                        FeatureSpec(dtype=DataType.INT8, name='PCLASS'),
		FeatureSpec(dtype=DataType.INT8, name='SIBSP'),
		FeatureSpec(dtype=DataType.INT8, name='PARCH'),
		FeatureSpec(dtype=DataType.DOUBLE, name='FARE')
                    ],
                    outputs=[
                        FeatureSpec(dtype=DataType.INT8, name='PCLASS'),
		FeatureSpec(dtype=DataType.INT8, name='SIBSP'),
		FeatureSpec(dtype=DataType.INT8, name='PARCH'),
		FeatureSpec(dtype=DataType.DOUBLE, name='FARE'),
		FeatureSpec(dtype=DataType.INT8, name='PREDICTIONS')
                    ]
                ), 'decision_function': ModelSignature(
                    inputs=[
                        FeatureSpec(dtype=DataType.INT8, name='PCLASS'),
		FeatureS

## Training RandomForestClassifier V2

### Feature Engineering Transformers

In [50]:
from snowflake.snowpark.dataframe import DataFrame, col
from snowflake.ml.modeling.preprocessing import OneHotEncoder
from snowflake.ml.modeling.impute import SimpleImputer
import snowflake.snowpark.functions as snow_func
from snowflake.snowpark.session import Session


def extract_salutations(session: Session, df: DataFrame) -> DataFrame:
    df.write.save_as_table(
        table_name='__temp_titanic_train',
        mode='overwrite',
        table_type='temporary'
    )
    return session.sql(
        "select *, regexp_substr(name, '\\\w+\\\.') as salutation from __temp_titanic_train"
    )

def drop_non_impact_cols(df: DataFrame) -> DataFrame:
    return df.drop(['PassengerId', 'Name', 'Ticket', 'Embarked'])

def cabin_booked(df: DataFrame) -> DataFrame:
    df_cabin = df.with_column(
        'Cabin',
        snow_func.when(col('Cabin').is_not_null(), 'Y').otherwise('N')
    )

    return df_cabin

def encode_one_hot(df: DataFrame, cat_cols: list, cat_cols_out: list) -> DataFrame:
    ohe = OneHotEncoder(
        input_cols=cat_cols,
        output_cols=cat_cols_out,
        drop_input_cols=True
    )

    ohe.fit(df)
    df_encoded = ohe.transform(df)

    return df_encoded

def fill_missing_values(df: DataFrame, columns: list) -> DataFrame:
    sim = SimpleImputer(
        strategy='most_frequent',
        input_cols=columns,
        output_cols=columns
    )

    sim.fit(df)
    df_imputed = sim.transform(df)

    return df_imputed

### Prepare Training Data

In [51]:
log.info("Extract salutations")
salutation_df = extract_salutations(session, training_input_df)

log.info("Dropping non impact columns")
dropped_col_df = drop_non_impact_cols(df=salutation_df)

log.info("Converting Cabin column to binary")
cabin_df = cabin_booked(dropped_col_df)

log.info("Encoding categorical columns")
cat_cols = ['sex', 'cabin', 'salutation']
cat_cols_out = [f'{c}_encoded' for c in cat_cols]
encoded_df = encode_one_hot(cabin_df, cat_cols, cat_cols_out)

log.info("Filling missing values")
cols_missing_values = ['AGE']
v2_training_df = fill_missing_values(encoded_df, cols_missing_values)
v2_training_df.show()

INFO:__main__:Extract salutations
INFO:__main__:Dropping non impact columns
INFO:__main__:Converting Cabin column to binary
INFO:__main__:Encoding categorical columns
INFO:__main__:Filling missing values


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"AGE"  |"sex_encoded_female"  |"sex_encoded_male"  |"cabin_encoded_N"  |"cabin_encoded_Y"  |"salutation_encoded_Capt."  |"salutation_encoded_Col."  |"salutation_encoded_Countess."  |"salutation_encoded_Don."  |"salutation_encoded_Dr."  |"salutation_encoded_Jonkheer."  |"salutation_encoded_Lady."  |"salutation_encoded_Major."  |"salutation_encoded_Master."  

### Prepare Test Data

In [52]:
log.info("Extract salutations")
salutation_df = extract_salutations(session, test_input_df)

log.info("Dropping non impact columns")
dropped_col_df = drop_non_impact_cols(df=salutation_df)

log.info("Converting Cabin column to binary")
cabin_df = cabin_booked(dropped_col_df)

log.info("Encoding categorical columns")
cat_cols = ['sex', 'cabin', 'salutation']
cat_cols_out = [f'{c}_encoded' for c in cat_cols]
encoded_df = encode_one_hot(cabin_df, cat_cols, cat_cols_out)

log.info("Filling missing values")
cols_missing_values = ['AGE']
v2_test_df = fill_missing_values(encoded_df, cols_missing_values)
v2_test_df.show()

INFO:__main__:Extract salutations
INFO:__main__:Dropping non impact columns
INFO:__main__:Converting Cabin column to binary
INFO:__main__:Encoding categorical columns
INFO:__main__:Filling missing values


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"AGE"  |"sex_encoded_female"  |"sex_encoded_male"  |"cabin_encoded_N"  |"cabin_encoded_Y"  |"salutation_encoded_Capt."  |"salutation_encoded_Col."  |"salutation_encoded_Countess."  |"salutation_encoded_Don."  |"salutation_encoded_Dr."  |"salutation_encoded_Jonkheer."  |"salutation_encoded_Lady."  |"salutation_encoded_Major."  |"salutation_encoded_Master."  

### Train the Model

In [53]:
V2_FEATURE_COLS = [c for c in v2_training_df.columns if not c.startswith("SURVIVED")]
LABEL_COLS = ["SURVIVED"]
OUTPUT_COLS = ["PREDICTIONS"]

v2_rf = RandomForestClassifier(
    input_cols=V2_FEATURE_COLS,
    label_cols=LABEL_COLS,
    output_cols=OUTPUT_COLS
)

v2_rf.fit(v2_training_df)

In [54]:
# Use the model to make predictions.
v2_rf_training_prediction_df = v2_rf.predict(v2_training_df)
v2_rf_training_prediction_df[LABEL_COLS[0], OUTPUT_COLS[0]].show()

------------------------------
|"SURVIVED"  |"PREDICTIONS"  |
------------------------------
|0           |0              |
|1           |1              |
|1           |1              |
|1           |1              |
|0           |0              |
|0           |0              |
|0           |0              |
|0           |0              |
|1           |1              |
|1           |1              |
------------------------------



### Calculate Prediction Accuracy on Training Data

In [55]:
from snowflake.ml.modeling.metrics import accuracy_score, confusion_matrix

v2_rf_training_accuracy_score = accuracy_score(
    df=v2_rf_training_prediction_df,
    y_true_col_names=LABEL_COLS,
    y_pred_col_names=OUTPUT_COLS
)
log.info(f"RandomForestClassifier v2 accuracy_score on training data: {v2_rf_training_accuracy_score}")

v2_rf_training_confusion_matrix = confusion_matrix(
    df=v2_rf_training_prediction_df,
    y_true_col_name=LABEL_COLS[0],
    y_pred_col_name=OUTPUT_COLS[0]
)
log.info(f"RandomForestClassifier v2 confusion_matrix on training data: {v2_rf_training_confusion_matrix}")

INFO:__main__:RandomForestClassifier v2 accuracy_score on training data: 0.986532
INFO:__main__:RandomForestClassifier v2 confusion_matrix on training data: [[546.   3.]
 [  9. 333.]]


### Logging the V2 Model

In [56]:
mv = reg.log_model(
    v2_rf,
    model_name="Titanic_RF",
    version_name="v2",
    pip_requirements=["packaging"],
    comment="Very simple RF model to setup baseline",
    metrics={
        "training_accuracy_score": v2_rf_training_accuracy_score,
        "training_confusion_matrix": v2_rf_training_confusion_matrix.tolist()
    },
    sample_input_data=v2_training_df
)

INFO:snowflake.ml.registry._manager.model_manager:Start packaging and uploading your model. It might take some time based on the size of the model.
INFO:absl:Model signatures are auto inferred as:

{'predict': ModelSignature(
                    inputs=[
                        FeatureSpec(dtype=DataType.DOUBLE, name='AGE'),
		FeatureSpec(dtype=DataType.DOUBLE, name='"sex_encoded_female"'),
		FeatureSpec(dtype=DataType.DOUBLE, name='"sex_encoded_male"'),
		FeatureSpec(dtype=DataType.DOUBLE, name='"cabin_encoded_N"'),
		FeatureSpec(dtype=DataType.DOUBLE, name='"cabin_encoded_Y"'),
		FeatureSpec(dtype=DataType.DOUBLE, name='"salutation_encoded_Capt."'),
		FeatureSpec(dtype=DataType.DOUBLE, name='"salutation_encoded_Col."'),
		FeatureSpec(dtype=DataType.DOUBLE, name='"salutation_encoded_Countess."'),
		FeatureSpec(dtype=DataType.DOUBLE, name='"salutation_encoded_Don."'),
		FeatureSpec(dtype=DataType.DOUBLE, name='"salutation_encoded_Dr."'),
		FeatureSpec(dtype=DataType.DOUBLE, name='"salu

# Use the Model Just Logged
The `log_model()` method returns a reference to the *model version* just got logged. It can be used make predictions.

In [57]:
v2_test_prediction_df = mv.run(v2_test_df, function_name='predict')
v2_test_prediction_df[LABEL_COLS[0], OUTPUT_COLS[0]].show()

------------------------------
|"SURVIVED"  |"PREDICTIONS"  |
------------------------------
|0           |0              |
|1           |1              |
|1           |1              |
|1           |1              |
|0           |0              |
|0           |0              |
|0           |0              |
|0           |0              |
|1           |1              |
|1           |1              |
------------------------------



In [58]:
v2_test_accuracy = accuracy_score(df=v2_test_prediction_df, y_true_col_names=LABEL_COLS, y_pred_col_names=OUTPUT_COLS)
log.info(f"RandomForestClassifier v2 accuracy on test data: {v2_test_accuracy}")

INFO:__main__:RandomForestClassifier v2 accuracy on test data: 0.986532


## Use A Model from Registry
The `show_models()` method returns a `Pandas` DataFrame

In [ ]:
models_df = reg.show_models()
models_df.head()

### Two Ways to Get A Model Reference/Instance from Registry
1. The `models()` method returns the list of all models (each being a `Model` instance)
2. `get_model(model_name)` returns an instance of the named model

In [65]:
reg.show_models()

,created_on,name,database_name,schema_name,comment,owner,default_version_name,versions
0,2024-02-09 09:04:04.762000-08:00,TITANIC_LINEARSVC,MACHINE_LEARNING,MODEL_REGISTRY,None,ROLE_SNOWPARK,V1,"[""V1""]"
1,2024-02-09 09:00:46.602000-08:00,TITANIC_RF,MACHINE_LEARNING,MODEL_REGISTRY,None,ROLE_SNOWPARK,V1,"[""V1"",""V2""]"


#### models()

In [59]:
model_list = reg.models()
[m.name for m in model_list]

['TITANIC_LINEARSVC', 'TITANIC_RF']

In [60]:
rf_model = model_list[1]
rf_model.versions()

In [61]:
rf_model.show_versions()

,created_on,name,comment,database_name,schema_name,module_name,is_default_version,functions,metadata,user_data
0,2024-02-09 09:00:46.635000-08:00,V1,Very simple RF model to setup baseline,MACHINE_LEARNING,MODEL_REGISTRY,TITANIC_RF,true,"[""PREDICT_PROBA"",""PREDICT"",""PREDICT_LOG_PROBA""]","{""metrics"": {""training_accuracy_score"": 0.8361...","{""snowpark_ml_data"":{""functions"":[{""name"":""PRE..."
1,2024-02-09 09:09:28.132000-08:00,V2,Very simple RF model to setup baseline,MACHINE_LEARNING,MODEL_REGISTRY,TITANIC_RF,false,"[""PREDICT_PROBA"",""PREDICT"",""PREDICT_LOG_PROBA""]","{""metrics"": {""training_accuracy_score"": 0.9865...","{""snowpark_ml_data"":{""functions"":[{""name"":""PRE..."


In [62]:
rf_mv1 = model_list[1].version('v1')
pred_df1 = rf_mv1.run(v1_test_df, function_name='predict')
pred_df1['SURVIVED', 'PREDICTIONS'].show()

------------------------------
|"SURVIVED"  |"PREDICTIONS"  |
------------------------------
|0           |0              |
|1           |1              |
|1           |0              |
|1           |1              |
|0           |0              |
|0           |0              |
|0           |0              |
|0           |0              |
|1           |1              |
|1           |0              |
------------------------------



#### get_model(model_name)

In [63]:
lsvc_model = reg.get_model("Titanic_LinearSVC")
lsvc_model.versions()

In [64]:
lsvc_mv1 = lsvc_model.version('v1')
pred_df1_lsvc = lsvc_mv1.run(v1_test_df, function_name='predict')
pred_df1_lsvc['SURVIVED', 'PREDICTIONS'].show()

------------------------------
|"SURVIVED"  |"PREDICTIONS"  |
------------------------------
|0           |0              |
|1           |1              |
|1           |0              |
|1           |1              |
|0           |0              |
|0           |0              |
|0           |1              |
|0           |0              |
|1           |0              |
|1           |0              |
------------------------------

